# Toxicity Detection Pipeline

The toxicity detection pipeline is designed to identify and classify toxic content in text data. 

+ The pipeline can be configured using YAML files, or programmatically as shown below.
+ There are different ways to run the pipeline: You can use the `detect_toxicity` function directly, or set up a Gradio web application as shown in the [README.md](https://github.com/debatelab/toxicity-detector/blob/main/README.md).

This notebook demonstrates **how to set up and run the pipeline using a Hugging Face API key**. There are several possibilities to execute this notebook. You can, for instance,

1. execute this notebook on Colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/debatelab/toxicity-detector/blob/master/toxicity_pipeline_intro.ipynb), or
2. execute this notebook locally in, for instance, [JupyterLab](https://jupyter.org/). The second option requires you to have Python installed installed on your machine.

## Prerequisites: Using Hugging Face API Key

In this notebook, we will use models served through Hugging Face that require authentication via an API key. 

1. If you don't have a Hugging Face account yet, you can create one for free at [huggingface.co/join](https://huggingface.co/join). 
2. After creating an account, you can generate an API key by navigating to your account settings at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) and clicking on "New token". Make sure to copy the generated token, as you will need it to authenticate your requests.

Alternatively, you can configure the pipeline to use another model. Please refer to the [example configuration YAML file](https://github.com/debatelab/toxicity-detector/blob/main/config/pipeline_config.yaml) for details on how to set up different models.

## Installing the required packages

In [ ]:
%pip install toxicity-detector

## Configuration of the pipeline

In [ ]:
from toxicity_detector import detect_toxicity
from toxicity_detector.config import PipelineConfig

pipeline_config = PipelineConfig(
    # We use the Llama-3.2-3B model via the Hugging Face API
    # This model is very small and only suitable for demonstration purposes.
    # For serious use cases, consider using a larger and newer model.
    used_chat_model='Llama-3.2-3B',
    local_base_path='.',
    result_data_path='result_data',
    log_path='logs',
    models={
        'Llama-3.2-3B': {
            'name': 'Llama-3.2-3B',
            'description': 'Llama-3.2-3B over together.ai',
            'base_url': 'https://router.huggingface.co/together/v1',
            'model': 'meta-llama/Llama-3.2-3B-Instruct-Turbo',
            # Passing the API key directly here for demonstration purposes. 
            # In practice, you should use environment variables an pass here the 
            # name of the variable that holds the key, e.g.,
            # 'api_key_name': '<YOUR_ENV_VARIABLE_NAME>',
            'api_key': '<YOUR_HUGGINGFACE_API_KEY>',
            'llm_chain': 'chat-chain',
            'model_kwargs': {
                'max_tokens': 1024,
                'temperature': 0.2,
            }
        },
    }
)

## Running the pipeline via `detect_toxicity`

The function `detect_toxicity` will run the toxicity detection pipeline on the provided input text, return the result and store it in the specified result data path (the YAML file that contains the result contains all information to reproduce the run and the full reasoning trace).

In [ ]:
from IPython.display import display, Markdown

input_text = 'Peter is dumn.'

result = detect_toxicity(
    input_text=input_text,  # The text to be analyzed
    user_input_source=None,  # Optional: identifier for the source of the input (e.g., 'chat', 'comment')
    toxicity_type='hatespeech',  # Type of toxicity analysis to perform (one of: 'hatespeech' or 'personalized_toxicity')
    context_info=None,  # Optional: additional context about the conversation or situation
    pipeline_config=pipeline_config,  # Configuration specifying model, paths, and behavior
    serialize_result=True,  # If True, saves the result to disk as YAML
)

# Display results as formatted markdown
analysis = result.answer['analysis_result']
contains_toxicity = result.answer['contains_toxicity']

# Map toxicity values to display strings
toxicity_display = {
    ToxicityAnswer.FALSE: '✅ NO',
    ToxicityAnswer.TRUE: '⚠️ YES',
    ToxicityAnswer.UNCLEAR: '❓ UNKNOWN/UNCLEAR'
}

contains_toxicity_str = toxicity_display.get(contains_toxicity, 'Answer could not be parsed')

markdown_output = f"""
## 🔍 Toxicity Analysis Results

### Input Text
> {input_text}

### Contains Toxicity
**{contains_toxicity_str}**

### Analysis Result
{analysis}
"""

display(Markdown(markdown_output))